In [5]:
import sqlite3
import pandas as pd
import re
from pyarrow import feather

task = 'CARIT-PREPOT'
id_list_files = [ 
    f"first_level/{task}-l1-list_{i}run.txt" 
    for i in range(1, 3) 
]
print(id_list_files)
df_list = []
for id_file in id_list_files:
    df_list.append(pd.read_table(id_file, sep=" ", header=None, names=["pid", "hcp_task", "fsf"]))
id_list = pd.concat(df_list, axis=0)
def parse_pid(pid):
    match = re.search(r"(?P<pid>HCD\d+)_(?P<session>V\d+)_MR", pid)

    if not match:
        return (None, None)

    pid = match.group("pid")
    session = match.group("session")

    return (match.group("pid"), match.group("session"))

id_list['pid'], id_list['session'] = zip(*id_list['pid'].apply(parse_pid))

# Connect to the SQLite database
conn = sqlite3.connect('hcpd_file_status.db')

task = "CARIT_PREPOT"
# Query the database to load the entire 'files' table into a DataFrame
df = pd.read_sql_query(f"""
SELECT * FROM files WHERE 
file_type IS 'txt' AND
task IS '{task}' AND
data_type IS 'Parcellated'
""", conn)

# Close the connection
conn.close()

['first_level/CARIT-PREPOT-l1-list_1run.txt', 'first_level/CARIT-PREPOT-l1-list_2run.txt']


In [6]:
df

,id,filepath,status,pid,session,task,data_type,file_type
0,99010,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0092334...,built,HCD0092334,V1,CARIT_PREPOT,Parcellated,txt
1,99011,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0086440...,built,HCD0086440,V1,CARIT_PREPOT,Parcellated,txt
2,99012,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0022919...,built,HCD0022919,V2,CARIT_PREPOT,Parcellated,txt
3,99013,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0119126...,built,HCD0119126,V1,CARIT_PREPOT,Parcellated,txt
4,99014,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0232223...,built,HCD0232223,V2,CARIT_PREPOT,Parcellated,txt
...,...,...,...,...,...,...,...,...
62841,161851,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0146937...,built,HCD0146937,V1,CARIT_PREPOT,Parcellated,txt
62842,161852,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0146937...,built,HCD0146937,V1,CARIT_PREPOT,Parcellated,txt
62843,161853,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0146937...,built,HCD0146937,V1,CARIT_PREPOT,Parcellated,txt
62844,161854,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0146937...,built,HCD0146937,V1,CARIT_PREPOT,Parcellated,txt


In [1]:
def has_missing(group):
    return 'missing' in group['status'].values
print(len(id_list[id_list.pid == 'HCD2562658']))
df_pid_missing_any = df.groupby(['pid', 'session']).apply(has_missing).reset_index(name='has_missing')
df_pid_missing_any = pd.merge(id_list, df_pid_missing_any, how='left', on=['pid', 'session'])
failed_to_extract_df = df_pid_missing_any[df_pid_missing_any['has_missing'].isna() | df_pid_missing_any['has_missing']]

['first_level/CARIT_PREPOT-l1-list_1run.txt', 'first_level/CARIT_PREPOT-l1-list_2run.txt']


FileNotFoundError: [Errno 2] No such file or directory: 'first_level/CARIT_PREPOT-l1-list_1run.txt'

In [4]:
!python3 -m pip install pandas tabulate